In [10]:
import random
import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, average_precision_score


# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Specify whether you want to use a bidirectional or unidirectional LSTM
bidirectional = True    

num_epochs = 10


In [11]:
def set_seed(seed):
    torch.manual_seed(seed)  # Sets the seed for CPU operations
    torch.cuda.manual_seed(seed)  # Sets the seed for CUDA GPU operations
    torch.cuda.manual_seed_all(seed)  # If using multiple GPUs
    random.seed(seed)  # Python's random library
    np.random.seed(seed)  # NumPy
    
    # For determinism in certain CUDA operations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [12]:
# Load
df_a = pd.read_parquet('../../data/set-a-imputed-scaled.parquet').sort_values(by=['RecordID','Time'])
df_b = pd.read_parquet('../../data/set-b-imputed-scaled.parquet').sort_values(by=['RecordID','Time'])
df_c = pd.read_parquet('../../data/set-c-imputed-scaled.parquet').sort_values(by=['RecordID','Time'])


# Get labels
outcomes_a = pd.read_csv('../../data/Outcomes-a.txt').sort_values(by=['RecordID'])[['RecordID', 'In-hospital_death']]
outcomes_b = pd.read_csv('../../data/Outcomes-b.txt').sort_values(by=['RecordID'])[['RecordID', 'In-hospital_death']]
outcomes_c = pd.read_csv('../../data/Outcomes-c.txt').sort_values(by=['RecordID'])[['RecordID', 'In-hospital_death']]


# Drop time
df_a = df_a.drop(columns=['Time'])
df_b = df_b.drop(columns=['Time'])
df_c = df_c.drop(columns=['Time'])


# Group by 'Category'
df_a = df_a.groupby('RecordID')
df_b = df_b.groupby('RecordID')
df_c = df_c.groupby('RecordID')

# Convert groups into stacked tensor (dropping the group column dynamically)
X_train = torch.stack([torch.tensor(group.drop(columns='RecordID').values, dtype=torch.float32) for _, group in df_a])  # Stack into a 3D tensor
X_val = torch.stack([torch.tensor(group.drop(columns='RecordID').values, dtype=torch.float32) for _, group in df_b])
X_test = torch.stack([torch.tensor(group.drop(columns='RecordID').values, dtype=torch.float32) for _, group in df_c])  

y_train = torch.tensor(outcomes_a.drop(columns=['RecordID']).values, dtype=torch.float32)
y_val = torch.tensor(outcomes_b.drop(columns=['RecordID']).values, dtype=torch.float32)
y_test = torch.tensor(outcomes_c.drop(columns=['RecordID']).values, dtype=torch.float32)


train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
val_dataset = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [13]:
# Define the LSTM classifier
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers, num_classes, dropout, bidirectional):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )
        if bidirectional:
            hidden_size = hidden_size * 2
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        lstm_out, _ = self.lstm(x)
        # We only want the output from the last time step
        last_time_step_out = lstm_out[:, -1, :]
        out = self.fc(last_time_step_out)
        out = self.sigmoid(out)
        return out

In [14]:
def objective(trial):
     hidden_size = trial.suggest_categorical("hidden_size", [64, 128, 256]) 
     num_layers = trial.suggest_categorical("num_layers", [1, 2, 4, 8])
     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True) # Learning rate
     weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)  # Weight decay
     dropout = trial.suggest_float('dropout', 0.1, 0.75)  # Weight decay


     model = LSTMClassifier(
          input_dim=41, 
          hidden_size=hidden_size, 
          num_layers=num_layers, 
          num_classes=1, 
          dropout=dropout, 
          bidirectional=bidirectional
     ).to(device)


     optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
     criterion = nn.BCELoss()

     best_val_score = 0.0
     for epoch in range(num_epochs):
          model.train()
          for inputs, targets in train_loader:
              inputs, targets = inputs.to(device), targets.to(device)
              optimizer.zero_grad()
              outputs = model(inputs)
              loss = criterion(outputs.squeeze(), targets.squeeze())
              loss.backward()
              optimizer.step()

          # Validation loop
          model.eval()
          all_val_targets = []
          all_val_predictions = []
          with torch.no_grad():
               for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    probabilities = model(inputs)
                    all_val_predictions.extend(probabilities.squeeze().cpu().tolist())
                    all_val_targets.extend(targets.cpu().tolist())

          # Calculate validation metrics
          val_auroc = roc_auc_score(all_val_targets, all_val_predictions)
          val_auprc = average_precision_score(all_val_targets, all_val_predictions)

          # Combine AUROC and AUPRC into a single score (weighted sum)
          combined_score = 0.5 * val_auroc + 0.5 * val_auprc  # Adjust weights as needed
          best_val_score = max(best_val_score, combined_score)

     return best_val_score  # Return the best combined score

In [15]:
set_seed(42)

# study = optuna.create_study(direction="maximize")  
# study.optimize(objective, n_trials=25)

# best_params = study.best_params

# # Print the best hyperparameters
# print("Best hyperparameters:", best_params)

In [16]:
if bidirectional:
    best_params = {
        'hidden_size': 256, 
        'num_layers': 4, 
        'learning_rate': 5.095326515363872e-05, 
        'weight_decay': 0.0005569056693796072, 
        'dropout': 0.17100380863595918
    }
else:
    best_params = {
        'hidden_size': 256, 
        'num_layers': 2, 
        'learning_rate': 9.892595905291616e-05, 
        'weight_decay': 1.4600838938197394e-05, 
        'dropout': 0.18640091013719517
    }



model = LSTMClassifier(
    input_dim=41, 
    hidden_size=best_params["hidden_size"], 
    num_layers=best_params["num_layers"], 
    num_classes=1, 
    dropout=best_params["dropout"], 
    bidirectional=bidirectional
).to(device)
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr=best_params['learning_rate'],
    weight_decay=best_params["weight_decay"]
)
criterion = nn.BCELoss()



# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("Training complete!")

Epoch [1/10], Loss: 0.3718
Epoch [2/10], Loss: 0.3077
Epoch [3/10], Loss: 0.2998
Epoch [4/10], Loss: 0.2927
Epoch [5/10], Loss: 0.2854
Epoch [6/10], Loss: 0.2777
Epoch [7/10], Loss: 0.2702
Epoch [8/10], Loss: 0.2627
Epoch [9/10], Loss: 0.2527
Epoch [10/10], Loss: 0.2471
Training complete!


In [17]:
# Evaluation loop
model.eval()  # Set model to evaluation mode
total_loss = 0
all_labels = []
all_probs = []

with torch.no_grad():  # Disable gradient computation for efficiency
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        total_loss += loss.item()

        # Get probabilities (if using softmax for multi-class or sigmoid for binary)
        probs = outputs


        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Compute metrics
average_loss = total_loss / len(test_loader)
auroc = roc_auc_score(all_labels, all_probs)
auprc = average_precision_score(all_labels, all_probs)

print(f"Test AuROC: {auroc:.4f}")
print(f"Test AuPRC: {auprc:.4f}")

Test AuROC: 0.8270
Test AuPRC: 0.4871
